In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np

import random
import sys
import io
import re

Using TensorFlow backend.


# Preprocess

In [25]:
path = '달빛조각사01.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

text = re.sub(r'<.*>', '', text)
text = re.sub(r'\n', ' ', text)
text = re.sub(r' +', ' ', text)

print('corpus length:', len(text))

corpus length: 153496


# Compute Character Indices

In [26]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 1131


# Vectorize Sentences

In [28]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


nb sequences: 51152
Vectorization...


# Create Simple Model

In [29]:
print('Build model...')
model = Sequential()
model.add(LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001))

Build model...


# Define Callback

In [32]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Train

In [33]:
model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

Epoch 1/1
51152/51152 [==============================] - 1606s 31ms/step - loss: 4.2690

----- Generating text after Epoch: 0
----- Generating with seed: "기기도 한다. - 투지 : 순간적인 괴력을 내기도 하고, 눈빛만으로도 약"
기기도 한다. - 투지 : 순간적인 괴력을 내기도 하고, 눈빛만으로도 약개 있었다. " " 마의 남을 감을 차 을 하는 차 이 하은 무가을 받은 3어 가 수 있었다. 위드는 조각 타이 고 있었다. "이 무각하서 만 한 것이 이 병들을 이 하을 레으로 수이 하기 않어 자 이 있어 내 고 있었다. " 네. 아이 트술을 동이 수으로 습어 가 들과 만 이 가 할 있는 진의 있금 이 보나 어 있었다. " 조각술 " 그직 인만 스킬들은 바현 있었다. " 직을 에 난을 지바를 사굴 에 생이 지 있을 전의 아은 있는 중어 사들을 마이 있도 이 요. 그이 제 이들의 정이다. 을 저은 때한 먹은 간 가 하고 있는 " 사 정집 하다. 다. 그렇  우이 조각 보지 그할 1면 있는 이 왕에 것이 있았다. " 이 자우 무개 면 미하다. 그 말은 할 이 하이 마은 예동 속 번 나은 움을 인지 있는 것이 
